In [2]:
from datasets import load_dataset

In [3]:
qa2d = load_dataset("domenicrosati/QA2D", split="train[:50000]")

In [4]:
qa2d = qa2d.train_test_split(test_size=0.1)

In [5]:
qa2d["train"][0]

{'dataset': 'SQuAD',
 'example_uid': '572fff11947a6a140053cf1e',
 'question': 'Which interstate is the only one that uses kilometer posts instead of mileposts ?',
 'answer': 'I-19',
 'turker_answer': 'I-19 is the only interstate that uses kilometer posts instead of mileposts .',
 'rule-based': 'The only one that uses kilometer posts instead of mileposts is i-19 .'}

In [6]:
def filter_dataset(example):
    """
    Make sure that the samples used for training
    1. Have an answer
    2. Have a question 
    3. Has an answer (turker_answer)
    """
    usable = example["answer"] != "" and example["question"] != "" and example["turker_answer"] != ""
    return usable

In [7]:
qa2d.filter(filter_dataset)

Filter:   0%|          | 0/45000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['dataset', 'example_uid', 'question', 'answer', 'turker_answer', 'rule-based'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['dataset', 'example_uid', 'question', 'answer', 'turker_answer', 'rule-based'],
        num_rows: 5000
    })
})

In [24]:
checkpoint = "google/flan-t5-base"

In [25]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

D:\MyProjects\DawsonAI\.env\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\hf_cache\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [26]:
def generate_targets(examples):
    # The output is of format "question? answer"
    return list(map(
        lambda q, a: f"{q} {a}",
        examples["question"],
        examples["answer"],
    ))

In [27]:
generate_targets(qa2d["train"][:3])

['Which interstate is the only one that uses kilometer posts instead of mileposts ? I-19',
 'What kind of topics began appearing more commonly in poetry and literature during the Enlightenment ? scientific topics',
 'What is a major reason that Bermuda has problems with overpopulation ? limited land area ,']

In [28]:
def preprocess_function(examples):
    """ 
    The objective of our model is to transform a sentence into a question.
    To do so, the input to the model will be the sentence itself.
    The output of the model is be the question followed by the answer.
    """
    prompt = "ask: " # Each input will be formatted as "ask: sentence..."
    inputs = [prompt + x for x in examples["turker_answer"]] 
    # The output is of format "question? answer"
    targets = generate_targets(examples)

    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [29]:
statements = qa2d["train"][20:24]
for sample in zip(statements["turker_answer"], generate_targets(statements)):
    print(f"{sample[0]} => {sample[1]}")

The author mainly wants to show the competitive advantages of Melbourne in the passage . => What does the author mainly want to show in the passage ? The competitive advantages of Melbourne .
Sony includes call Of Duty 3 in their low-end price range . => Which Call of Duty title does Sony include in their low-end price range ? Call Of Duty 3
People communicate with dogs by voice commands , body language or posture and hand signals . => People communicate with dogs by voice commands , body language or posture and what else ? hand signals
Lew Wasserman was the head of MCA in 1990 . => Who was the head of MCA in 1990 ? Lew Wasserman


In [30]:
tokenized_qa2d = qa2d.map(preprocess_function, batched=True)

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [31]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [32]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [37]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [38]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_qa2d_t5_flan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=4,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_qa2d["train"],
    eval_dataset=tokenized_qa2d["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [69]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

mcq_model = AutoModelForSeq2SeqLM.from_pretrained("results_qa2d/checkpoint-11252")

In [80]:
# Don't forget to add the "ask: " prompt!
input_text = "ask: The chair conformation of cyclochexane is more stable than its boat conformation."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = mcq_model.generate(input_ids, max_new_tokens=100, do_sample=False)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

What is the chair conformation of cyclochexane more stable than its boat conformation?
